<a href="https://colab.research.google.com/github/odu-cs-580-f21/assignment_5/blob/master/README.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CS 480/580 - Assignment_5 - Dániel B. Papp

# Notice

Since I wasn't able to use the dataset provided in the assignment document, I used the same dataset but provided by Kaggle in a `.csv` format. The data and the Kaggle challenge can be found [here](https://www.kaggle.com/c/digit-recognizer/data).

# How to run

```console
python3 app.py
```

# How it works

On a high level, the program reads the `.csv` dataset and makes predictions for each image. The predictions are then compared to the actual labels and the accuracy is calculated. The accuracy is then printed to the terminal.

All the necessary mathematical operations will be explained function by function below.

In [13]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

data = pd.read_csv('./train.csv')
data = np.array(data)

# Neural Network (NN) structure overview

Our NN will have three layers in total. To further improve the accuracy of our model, we could increase the number or layers or the number of nodes in the hidden layer.

1. Input layer
   - The input layer has 784 nodes, which corresponds with the 28x28 pixels of the image. Each individual pixel is normalized to a value between 0 and 1. Since we only care about the pixel either being `rgb(0,0,0)` or `rgb(255,255,255)`, we can normalize each pixel to represent true or false (1 or 0) weather it is black or white.
2. Hidden layer
   - For the sake of simplicity, we will use a single hidden layer with 10 nodes. The value of each node is calculated based on weights and biases applied to the value of the 784 nodes in the input layer.
3. Output layer
   - The output layer has 10 nodes, each representing a digit from 0 through 9. The value of each node is calculated based on weights and biases applied to the value of the 10 nodes in the hidden layer using a softmax activation algorithm.

In [14]:
m, n = data.shape
np.random.shuffle(data) 

data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255.
_,m_train = X_train.shape

# The mathematical operations and functions

## Our data

Each of our training example can be represented as a vector of 784 values. These vectors are then stacked into a matrix so we can calculate error from all examples at once with matrix operations.

Our matrix will have an $m {\times} n$ dimension, where m is the number of training examples and n is the number of nodes in the input layer (784). We transpose the matrix so the dimensions will be $n {\times} m$, with each column corresponding to a training example and each row is a node.

## Weight and biases

Between every two layers is a set of connections between every node in the previous layer and every node in the following layer. Which means there is a weight of of $n^{[l]}$ for every `i` in the number of nodes in the previous layer and every `j` in the number of nodes in the following layer.

From this we can conclude that our weights as a matrix will be $n^{[l]}{\times}n^{[l - 1]}$ where $n^{[l]}$ is the number of nodes in the previous layer and $n^{[l - 1]}$ is the number of nodes in the following layer. We call this matrix the $W^{[l]}$ matrix corresponding to layer ${l}$ of our network.

- $W^{[1]}$ will be represneting a $10 {\times} 784$ matrix, taking 784 nodes from the input layer corresponding to 10 nodes in the hidden layer.
- $W^{[2]}$ will have the dimensions of $10{\times}10$

In [15]:
def initParams():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

## Forward propagation

The first part of the function that is needed to calculate the forward propogation is the unactived values of the nodes in the first hidden layer. We can calculate that by applying $W^{[1]}$ and $b^{[1]}$ to our input layer. 

This produces $Z^{[1]} = W^{[1]} X + b^{[1]}$ where $X$ has the dimentions of $784{\times}m$, and $W^{[1]}$ has the dimentions of $10{\times}784$.

$W^{[1]} X$ is the dot product between the two, yielding a new matrix of dimensions $10{\times}m$. 

Our bias term $b^{[1]}$ has dimensions of $10{\times}1$, but our goal is to apply the same column of bias to all $m$ columns of traning data. This means that $b^{[1]}$ is effectivly $10{\times}m$ when calculating $Z^{[1]}$, which matches the dimensions of $W^{[1]} X$. 

### `ReLU(x)` (Rectified Linear Unit) activation function

This function is a simple non-linear activation function that we apply to $Z^{[1]}$, before it goes to our next layer. 

This function is linear when the input value is above 0, and outputs 0 otherwise. This is enough to prevent our model from turning into a linear regression model. 

$$A^{[1]} = {\text{ReLU}}(Z^{[1]}))$$

The output of this function will be called $A^{[1]}$, which holds the values of the nodes in the hidden layer of our NN after applying our activation function to it. 


In [16]:
def ReLU(Z):
    return np.maximum(Z, 0)

After calculating $A^{[1]}$ we can proceed to calculate the values for our output layer. We call the result of that function $Z^{[2]}$. 

$$Z^{[2]} = W^{[2]} A^{[1]} + b^{[2]}$$

Since $Z^{[2]}$ is our output layer, we need to apply our `softmax(x)` activation function to it. 

### `softmax(x) activation function`

This function takes a column of data at a time, taking each element in the column and outputting the exponential of that element divided by the sum of the exponentials of each of the elements in the input column. The result is a float value representing our probability (between 0, 1).

$$\frac {e^{z_i}}{{\sum^K_{j=1}e^{z_j}}} $$

The result of this function will be called $A^{[2]}$.

$$A^{[2]} = {\text{softmax}}(Z^{[2]})$$



In [17]:
def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

And with this completed, our forward propagation is completed. 

In [18]:
def forwardProp(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2



---



## Backward Propagation

The prupose of this function is to nudge our parameters to carry out gradient descent. Mathematically we are calulating the derivative of the loss function with respect to each weight and bias parameter. 

$$
J(\hat{y}, y) = -{\sum^c_{i=0}}y_i log(\hat{y}_i)
$$

### Cross-Entropy loss function

This is a helper function that classifies the products that result from the softmax activation function. In our function $\hat{y}$ represents our prediction vector, which looks like this:

$$
\begin{bmatrix}
  0.01 \\
  0.02 \\
  0.05 \\
  0.02 \\
  0.80 \\
  0.01 \\
  0.01 \\
  0.00 \\
  0.01 \\
  0.07 \\
\end{bmatrix}
$$

### Derivative `ReLU(x)` activation function

$$g^{[1]'}(Z^{[1]})$$

Wehn the input value is greater than 0, the function is linear with a derivative of 1. When the input value is less than 0, the function is horizontal with a derivative of 0. 

Thus the result of this function is a matrix of 1s and 0s based on the values of $Z^{[1]}$.

In [19]:
def ReLUDeriv(Z):
    return Z > 0

### One-Hot encoding function

In our backward propagation function, this is reprensted as $y$. This function correctly labels our training data while remaining a vector. The result in a example where our training node is 4, the one-hot encoding would return a vector like this:

$$
\begin{bmatrix}
  0 \\
  0 \\
  0 \\
  0 \\
  1 \\
  0 \\
  0 \\
  0 \\
  0 \\
  0 \\
\end{bmatrix}
$$

Notice that based on our backward propagation function $y_i = 0$ for all $i$ except the correct label, in our example above it is 4. Which means that the loss for the given example is just the log of the probability given for the correct prediction. 

In our example above $J(\hat{y}, y) = -\log({y}_4) = -\log(0.80) \approx 0.097$.

We can notice that the closer the prediction probability is to 1, the closer the loss is to 0, and in reverse, as the probability approaches 0, the loss function approaches $+\infty$. 


In [20]:
def oneHot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

### Maximizing cost function

By maximizing the cost function, we improve the accuracy of our model. We do so by substrcting the derivative of the loss function with respect to eacch parameter from that parameter over many rounds of graident descent. 

$$
W^{[1]} := W^{[1]} - \alpha \frac{{\partial}J}{{\partial}W^{[1]}}
$$

$$
b^{[1]} := b^{[1]} - \alpha \frac{{\partial}J}{{\partial}b^{[1]}}
$$

$$
W^{[2]} := W^{[2]} - \alpha \frac{{\partial}J}{{\partial}W^{[2]}}
$$

$$
b^{[2]} := b^{[2]} - \alpha \frac{{\partial}J}{{\partial}b^{[2]}}
$$

In [21]:
def updateParams(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

Since our goal with backward propagation is to find 
${\frac{\partial J}{\partial W^{[1]}}}$, ${\frac{\partial J}{\partial b^{[1]}}}$, ${\frac{\partial J}{\partial W^{[2]}}}$, and ${\frac{\partial J}{\partial b^{[2]}}}$, we have to first take a step backward in our NN layer and find ${\frac{\partial J}{\partial A^{[2]}}}$. For the sake of simplicity, we will write these derivatives as $dW^{[1]}$, $db^{[1]}$, $dW^{[2]}$, $db^{[2]}$, and finally $dA^{[2]}$. This derivative is simply equal to $dA^{[2]} = Y - A^{[2]}$.

From $dA^{[2]}$ we can calculate $dW^{[2]}$ and $db^{[2]}$ as follows:

$$
dW^{[2]} = \frac{1}{m}dZ^{[2]}A^{[1]T}
$$

$$
db^{[2]} = \frac{1}{m}{\sum}dZ^{[2]} 
$$

From here our goal is to calculate $dW^{[1]}$, and $db^{[1]}$ but first we need $dZ^{[1]}$. We find that using this formula:

$$
dZ^{[1]} = W^{[2]T}dZ^{[2]}. \times g^{[1]'}(Z^{[1]})
$$

_In our formula_ $g^{[1]'}(Z^{[1]})$ _stands for our_ `DerivativeReLU(x)` _activation function._

From here we can find $dW^{[1]}$, and $db^{[1]}$ by plugging $X$ in the formula instead of $A^{[1]}$. It will look like this:

$$
dW^{1]} = \frac{1}{m}dZ^{[1]}X^{[1]T}
$$

$$
db^{[1]} = \frac{1}{m}{\sum} dZ^{[1]} 
$$

After completing this, our `backwardProp()` function doesn't do anything else but we still need to update our values using the `updateParams()` function. If we take a look at the `gradientDescent()` function, we can see that after each iteration we update the params.  

In [22]:
def backwardProp(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = Y.size
    one_hot_Y = oneHot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLUDeriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

# Program logic

Below we are going to talk about each function that isn't math based but is used for our NN. _As these functions don't do any of the calculations, and only handle the business logic there isn't too much to say._ 

## `getPredictions()`

This function takes the return value of our forward propogation `softmax()` function, which we represented as $A^{[2]}$. It returns the index of our predicted digit. 

In [ ]:
def getPredictions(A2):
    return np.argmax(A2, 0)

## `getAccuracy()`

This function takes our prediction and our raw data as parameters. It prints the prediction and the layer and returns the accuracy calculated. 

In [ ]:
def getAccuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size

## `gradientDescent()`

This is our main function that wraps everything up together. It takes our training and testing data, our learning rate, and the number of iterations to complete. After every iteration it calls the `forwardProp()` function and proceeds to call the `backwardProp()` function. The inner workings of these functions was described above. After each iteration, the for loop updates our params considering our learning rate. 

After every 10th iteration, the program prints it's prediction and the accuracy of the prediction. 

In [ ]:
def gradientDescent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = initParams()
    # for every iteration desired by the function argument
    for i in range(iterations):
        Z1, A1, Z2, A2 = forwardProp(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backwardProp(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = updateParams(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        # every 10th iteration, print accuracy
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = getPredictions(A2)
            print(getAccuracy(predictions, Y))
    return W1, b1, W2, b2

The `gradientDescent()` function can be used as follows. It takes our X and Y NN data, our learning rate, and our iteration count.

In [ ]:
W1, b1, W2, b2 = gradientDescent(X_train, Y_train, 0.10, 500)

We decided to save $W^{[1]}$, $b^{[1]}$, $W^{[2]}$, and $b^{[2]}$ so we can make our predictions accordingly. 

In [ ]:
def makePredictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forwardProp(W1, b1, W2, b2, X)
    predictions = getPredictions(A2)
    return predictions

As we only care about the value of $A^{[2]}$ in this case, we discard other return values from the tuple. Then we proceed to input $A^{[2]}$ in our `getPredictions()` function, and return the resulted prediction. 

In [ ]:
def testPrediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = makePredictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

We can test the predictions made by our NN by supplying the index that we are interested in (in the range of 0, 9), and our $W^{[1]}$, $b^{[1]}$, $W^{[2]}$, and $b^{[2]}$ values that were calculated by the `gradientDescent()` function. 

In [ ]:
testPrediction(3, W1, b1, W2, b2)

# References

I used [this](https://www.youtube.com/watch?v=9RN2Wr8xvro) video to help we have a better understanding of how this digit recognition NN would be constructed. 